In this notebook we are working with a dataset of IMDB reviews. We heavily make use of the tutorial given with the competition. We want to perform sentiment analysis i.e. is a given review positive or negative? 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing the libraries we need. 

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('brown')
from nltk.corpus import stopwords
from collections import Counter 

In [ ]:
df_train = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', compression='zip',header=0,delimiter='\t',quoting=0, doublequote=False, escapechar='\\')

In [ ]:
df_train.shape

The training data has 3 columns: id, sentiment and review. 

In [ ]:
df_train.head(5)

# Data Cleaning and Text Preprocessing

Let's look at a few reviews. It will give us some insight on what needs to be cleaned up in the text. 

In [ ]:
df_train['review'][0]

In [ ]:
df_train['review'][1]

In [ ]:
df_train['review'][2]

A few things we observe: HTML tags, and both upper- and lower-case are used. Also, there are still some backslashes appearing before some possession apostrophes. 

1) We will use the BeautifulSoup package to remove the HTML tags as this is good practice compared to using a regular expressions. 

2) We will remove all punctuation. Although text like !!! and :( do convey sentiment, since this is our first attempt at sentiment analysis, for simplicity, we will remove all punctuation. However, we will treat numbers as words and retain them.

3) We will get rid of stop words. These are words that are used very often but do not convey any important information e.g. of, to, in, on etc. 

4) We will normalize the text by converting everything to lower-case.

In [ ]:
def review_to_words(review):
    html_removed = BeautifulSoup(review).get_text()
    punctuation_removed = re.sub('[^a-zA-Z0-9]',' ',str(html_removed))
    lower_case = punctuation_removed.lower()
    words = lower_case.split()
    #Searching sets in python is much faster than searching lists
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops] 
    return( " ".join(words ))

In [ ]:
processed_review = review_to_words( df_train['review'][0] )
print(df_train['review'][0])
print(processed_review)

In [ ]:
processed_reviews = []
for i in range(0,df_train['review'].size):
    processed_reviews.append(review_to_words(df_train['review'][i]))
    

# Using Bag of Words to create Features 

How many total distinct words are we left with after preprocessing? 

In [ ]:
words = []

for review in processed_reviews:
    for w in review.split():
        words.append(w)

print(words)

In [ ]:
print(len(words))
print(len(set(words)))

There are over 75000 unique words, therefore we will use only the most frequent 5000 words. 